In [ ]:
from loica import *
from flapjack import *

import matplotlib.pyplot as plt
import numpy as np
import getpass
import datetime
import random as rd
import pandas as pd

from numpy.fft import fft, ifft, fftfreq
from scipy.interpolate import interp1d, UnivariateSpline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import mean_absolute_error

from scipy.signal import savgol_filter, medfilt

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 6})

In [ ]:
import seaborn as sns

In [ ]:
def brownian_profile(t0, tmax, nt, sigma):
    steps = np.random.lognormal(size=(nt,), sigma=sigma)
    profile = np.cumprod(steps)
    profile = savgol_filter(profile, 21, 2)
    profile = profile / profile.max()
    t = np.linspace(t0, tmax, nt)
    return interp1d(t, profile)

def random_profile(t0, tmax, nt, fmax):
    freqs = fftfreq(nt)
    tff = np.zeros((nt,), dtype=np.complex)
    ncomps = len(freqs[np.abs(freqs)<fmax])
    tff[np.abs(freqs)<fmax] = np.random.normal(size=(ncomps,)) + np.random.normal(size=(ncomps,))*1j
    profile = ifft(tff).real
    profile = (profile - profile.min()) / (profile.max() - profile.min())
    profile = profile + np.random.uniform(-profile.min(), 1)
    profile = profile / profile.max()
    #profile = (profile - profile.min()) / (profile.max() - profile.min())
    t = np.linspace(t0, tmax, nt)
    return interp1d(t, profile)

def spline_profile(t0, tmax, nst):
    st = np.linspace(t0, tmax, nst)
    y = np.random.uniform(size=st.shape)
    profile = UnivariateSpline(st, y-y.min(), s=0)
    return profile

def gaussian_profile(t0, tmax, nt, n_blobs):
    t = np.linspace(t0, tmax, nt)
    profile = np.zeros_like(t)
    means = np.linspace(t.min(), t.max(), n_blobs)
    vars = [t.max()/n_blobs]*n_blobs #res.x[:n_blobs]
    heights = np.random.lognormal(size=(n_blobs,))
    for mean,var,height in zip(means, vars, heights):
        gaussian = height * np.exp(-(t-mean)*(t-mean) / var / 2) / np.sqrt(2 * np.pi * var)
        profile += gaussian
    return interp1d(t, profile/profile.max())

def growth_rate_profile(t):
    B0 = 0.01
    mu_max = np.random.uniform(0.5, 1)
    Bmax = 1
    l = np.random(0, 4)
    return gompertz_growth_rate(t, B0, Bmax, mu_max, l)

def growth_rate(t):
    B0 = 0.01
    mu_max = 1
    Bmax = 1
    l = 1
    return gompertz_growth_rate(t, B0, Bmax, mu_max, l)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 0.5)

In [ ]:
plt.figure(figsize=(1,1), frameon=False)
t = np.linspace(0, 24, 100)
for i in range(1):
    plt.plot(biomass(t), '-')
plt.ylim([0,1.1])
plt.xticks([])
plt.yticks([])
plt.savefig('growth_thumb.png', dpi=300)

In [ ]:
plt.figure(figsize=(1,1), frameon=False)
t = np.linspace(0, 24, 100)
for i in range(1):
    plt.plot(brownian_profile(0, 24, 100, 0.25)(t), '-')
plt.ylim([0,1])
plt.xticks([])
plt.yticks([])
plt.savefig('profile_thumb.png', dpi=300)

In [ ]:
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
#Defined data structure
columns = { 
    'Profile':[], 
    'nsr':[], 
    'Iteration':[], 
    'Method':[], 
    'Metric':[], 
    'Score':[], 
    'Data_true_profile':[],
    'Data_method_profile':[] 
}
df_characterization = pd.DataFrame(columns)

In [ ]:
profiles = ['brownian'] #, 'growth_rate', 'random', 'gaussian']
methods = ['indirect', 'direct', 'loica']
nsr = [0, 1e-4, 1e-3]
metrics = ['MAE', 'MSE']
#metrics = ['explained variance', 'MAE', 'MSE', 'mean_poisson', 'mean_gamma']

In [ ]:
study = fj.get('study', name='Method comparison')
fj.delete('study', study.id[0])

In [ ]:
#DNA and Vector are created inside the loop
metab = SimulatedMetabolism(biomass, growth_rate)

sfp1 = fj.get('signal', name='SFP0')
if len(sfp1)==0:
    sfp1 = fj.create('signal', name='SFP0', description='Simulated FP', color='green')
sfp1 = Reporter(name='CFP', degradation_rate=0, init_concentration=0, signal_id=sfp1.id[0])

media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
    
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')
    
biomass_signal = fj.get('signal', name='OD')

study = fj.get('study', name='Method comparison')
if len(study)==0:
    study = fj.create('study', name='Method comparison', description='Testing')

In [ ]:
samples = []
profile_timeseries = []

for n in nsr:
    for p in profiles:
        print(f'Profile: {p}, NSR: {n}')

        #iterar por 100 dnas y anadirlos a samples
        for i in range(100):
            #definir nobres para dnas
            name = 'Const_%s_nsr_%f_it_%i' % (p,n,i)
            dna = fj.get('dna', name=name)
            if len(dna)==0:
                dna = fj.create('dna', name=name)
            vector = fj.get('vector',name=name)    
            if len(vector)==0:
                vector = fj.create('vector', name=name, dnas=dna.id)

            #create the profile
            if p == 'random':
                profile = random_profile(t0=0, tmax=24, nt=100, fmax=0.05)
            elif p == 'spline':
                profile = spline_profile(t0=0, tmax=24, nst=20)
            elif p == 'gaussian':
                profile = gaussian_profile(t0=0, tmax=24, nt=100, n_blobs=20) # de que depende n_blobs?
            elif p=='growth_rate':
                profile = growth_rate
            elif p=='brownian':
                profile = brownian_profile(t0=0, tmax=24, nt=100, sigma=0.25)
            profile_timeseries.append(profile)
            
            #creation of LOICA model
            #GeneticNetwork
            const = GeneticNetwork(vector=vector.id[0])
            const.add_reporter(sfp1)
            const.add_operator(Source(output=sfp1, rate=1, profile=profile))
            #Assay
            for _ in range(1):
                sample = Sample(circuit=const, 
                                metabolism=metab,
                                media=media.id[0],
                                strain=strain.id[0])
                samples.append(sample)
               
    print(f'Creating assay for NSR {n}')
    assay = Assay(samples, 
          n_measurements=100, 
          interval=0.24,
          name=f'Loica constitutive expression NSR = {n}',
          description='Simulated constitutive gene generated by loica',
          biomass_signal_id=biomass_signal.id[0]
         )
    #Run and upload Assay
    print(f'Running simulation for NSR {n}')    
    assay.run(nsr=n)
    print(f'Uploading data for NSR {n}')
    assay.upload(fj, study.id[0])


In [ ]:
cfp = fj.get('signal', name='SFP0')
t = np.linspace(0, 24, 100)
pi = 0
for n in nsr:
    for p in profiles:
        for i in range(100):
            print('Characterizing ', p, n, i)
            profile = profile_timeseries[pi]
            pi += 1
            
            #get dnas
            name = 'Const_%s_nsr_%f_it_%i' % (p,n,i)
            vector = fj.get('vector',name=name)    
                
            #Characterize using LOICA
            source = Source(None, 0) #const.operators[0]
            source.characterize(
                fj,
                vector=vector.id,
                media=media.id,
                strain=strain.id,
                signal=cfp.id,
                biomass_signal=biomass_signal.id,
                n_gaussians=20,
                epsilon=0
            )
            
            #Characterize using indirect method
            er_indirect = fj.analysis(media=media.id, 
                    strain=strain.id,
                    vector=[vector.id[0]],
                    type='Expression Rate (indirect)',
                    biomass_signal=biomass_signal.id,
                    eps_L=1e-6,
                    pre_smoothing=11,
                    post_smoothing=11,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False
                      )
            #Characterize using direct method
            er_direct = fj.analysis(media=media.id, 
                    strain=strain.id,
                    vector=[vector.id[0]],
                    type='Expression Rate (direct)',
                    biomass_signal=biomass_signal.id,
                    eps_L=1e-5,
                    pre_smoothing=11,
                    post_smoothing=11,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False
                      )
            
            #Profiles
            true_profile = profile(t)
            indirect = er_indirect[er_indirect.Signal=='SFP0'].groupby('Time').Rate.mean().values
            direct = er_direct[er_direct.Signal=='SFP0'].groupby('Time').Rate.mean().values
            LOICA_profile = source.profile(t) * source.rate
            
            #Metrics
            #MSE
            mse_indirect = mean_squared_error(true_profile[1:], indirect)
            mse_direct = mean_squared_error(true_profile[1:], direct)
            mse_gaussian = mean_squared_error(true_profile[1:], LOICA_profile[1:])
            temp1 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Indirect', 'Metric':'MSE', 'Score':mse_indirect,'Data_true_profile':true_profile, 'Data_method_profile':indirect }  
            temp2 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Direct', 'Metric':'MSE', 'Score':mse_direct, 'Data_true_profile':true_profile, 'Data_method_profile':direct}  
            temp3 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Gaussian', 'Metric':'MSE', 'Score':mse_gaussian, 'Data_true_profile':true_profile, 'Data_method_profile':LOICA_profile}
            df_characterization = df_characterization.append([temp1, temp2, temp3], ignore_index=True)

            #MAE
            mae_indirect = mean_absolute_error(true_profile[1:], indirect)
            mae_direct = mean_absolute_error(true_profile[1:], direct)
            mae_gaussian = mean_absolute_error(true_profile[1:], LOICA_profile[1:])
            temp1 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Indirect', 'Metric':'MAE', 'Score':mae_indirect,'Data_true_profile':true_profile, 'Data_method_profile':indirect }  
            temp2 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Direct', 'Metric':'MAE', 'Score':mae_direct, 'Data_true_profile':true_profile, 'Data_method_profile':direct}  
            temp3 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Gaussian', 'Metric':'MAE', 'Score':mae_gaussian, 'Data_true_profile':true_profile, 'Data_method_profile':LOICA_profile}
            df_characterization = df_characterization.append([temp1, temp2, temp3], ignore_index=True)

            #Poisson deviance
            '''
            mpd_indirect = mean_poisson_deviance(true_profile[1:], indirect)
            mpd_direct = mean_poisson_deviance(true_profile[1:], direct)
            mpd_gaussian = mean_poisson_deviance(true_profile[1:], LOICA_profile[1:])
            temp1 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Indirect', 'Metric':'MPD', 'Score':mpd_indirect,'Data_true_profile':true_profile, 'Data_method_profile':indirect }  
            temp2 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Direct', 'Metric':'MPD', 'Score':mpd_direct, 'Data_true_profile':true_profile, 'Data_method_profile':direct}  
            temp3 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Gaussian', 'Metric':'MPD', 'Score':mpd_gaussian, 'Data_true_profile':true_profile, 'Data_method_profile':LOICA_profile}
            df_characterization = df_characterization.append([temp1, temp2, temp3], ignore_index=True)

            #gamma deviance

            mgd_indirect = mean_gamma_deviance(true_profile[1:], indirect)
            mgd_direct = mean_gamma_deviance(true_profile[1:], direct)
            mgd_gaussian = mean_gamma_deviance(true_profile[1:], LOICA_profile[1:])
            temp1 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Indirect', 'Metric':'MGD', 'Score':mse_indirect,'Data_true_profile':true_profile, 'Data_method_profile':indirect }  
            temp2 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Direct', 'Metric':'MGD', 'Score':mse_direct, 'Data_true_profile':true_profile, 'Data_method_profile':direct}  
            temp3 = {'Profile':p, 'nsr':n, 'Iteration':i, 'Method':'Gaussian', 'Metric':'MGD', 'Score':mse_gaussian, 'Data_true_profile':true_profile, 'Data_method_profile':LOICA_profile}
            df_characterization = df_characterization.append([temp1, temp2, temp3], ignore_index=True)
            '''
            
            
            
            
            

        
            
            
            #sometimes random profiles give 0 or negative valuea
            #characterize it
            #get metrics

In [ ]:
plt.plot(profile(t))

In [ ]:
df_characterization.to_json('df_characterization_100_brownian.json')

In [ ]:
df_plot[df_plot.Profile=='growth_rate']

In [ ]:
df_plot.Iteration.unique()

In [ ]:
df_plot = df_characterization[df_characterization['Metric']=='MSE'] #should also select metric in future

In [ ]:
plt.figure(dpi=300)
ax = sns.violinplot(x="Method", y="Score", data=df_plot, inner="stick", hue='nsr')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(2,2))
data = df_plot[df_plot.Method!='Indirect']
sns.barplot(
    ax=ax,
    x="Method", 
    y="Score", 
    data=data, 
    hue='nsr',
    color='gray',
    capsize=0.1,
    errwidth=2,
    #col="Profile", 
    #kind='bar', 
    #sharey=False,
    #figsize=(3,3)
)
plt.setp(ax.patches, linewidth=1, edgecolor='k')
plt.tight_layout()
plt.savefig('error_bar_graph.png', dpi=300)

In [ ]:
df_plot[df_plot.Profile=='gaussian'][df_plot.nsr==0.001]
print(nsr)

In [ ]:
from scipy.stats import ttest_ind
a = df_plot[df_plot.Method=='Gaussian'][df_plot.nsr==0][df_plot.Profile=='brownian']
b = df_plot[df_plot.Method=='Direct'][df_plot.nsr==0][df_plot.Profile=='brownian']
print(ttest_ind(a.Score,b.Score, equal_var=False))
print(a.Score.mean())
print(b.Score.mean())

In [ ]:
df = pd.read_json('df_characterization_100_brownian.json')

In [ ]:
t = np.linspace(0, 24, 100)
fig,axs = plt.subplots(4, 4, figsize=(2.5,2.5), sharex=True, sharey=True)
its = -np.arange(16) - 1 #np.random.randint(10, size=(16,))
for ax,it in zip(axs.ravel(), its):
    gaussian= df[(df.Method=='Gaussian')&(df.Metric=='MSE')&(df.Profile=='brownian')&(df.nsr==0)]
    gaussian = gaussian.sort_values('Score')
    iteration = gaussian.Iteration.values[it] # np.random.randint(10)
    gaussian = gaussian[gaussian.Iteration==iteration]
    gaussian_profile = gaussian.Data_method_profile.values[0]
    
    direct = df[(df.Method=='Direct')&(df.Metric=='MSE')&(df.Profile=='brownian')&(df.nsr==0)]
    direct = direct[direct.Iteration==iteration]
    direct_profile = direct.Data_method_profile.values[0]

    gtrue_profile = gaussian.Data_true_profile.values[0]
    dtrue_profile = direct.Data_true_profile.values[0]

    ax.plot(t[:-1], direct_profile, 'r', linewidth=1)
    ax.plot(t, gaussian_profile, 'g', linewidth=1)
    ax.plot(t, gtrue_profile, 'k--', linewidth=1)
    #ax.plot(dtrue_profile, 'b--', linewidth=1)
    ax.set_ylim([0,1])
    ax.set_xticks([])
    ax.set_yticks([])
#plt.tight_layout()
plt.savefig('profiles_worst_nsr_0.png', dpi=300)

In [ ]:
direct

### Here ends the first part

### Receiver

Create a genetic network and associate it with a Flapjack vector:

In [ ]:
dna = fj.get('dna', name='Rec_1e-3_nsr')
if len(dna)==0:
    dna = fj.create('dna', name='Rec_1e-3_nsr')
vector = fj.get('vector', name='Rec_1e-3_nsr')    
if len(vector)==0:
    vector = fj.create('vector', name='Rec_1e-3_nsr', dnas=dna.id)
    
rec = GeneticNetwork(vector=vector.id[0])

Create a reporter and associate it with a Flapjack signal so we can record the behaviour of the circuit:

In [ ]:
cfp = fj.get('signal', name='CFP')
sfp1 = Reporter(name='CFP', degradation_rate=0, init_concentration=1, signal_id=cfp.id[0])

rec.add_reporter(sfp1)

Create and add a receiver operator to the circuit, linking it to an AHL supplement:

In [ ]:
ahl = Supplement(name='AHL')
def sin_profile(t):
    return 1 - np.cos(2 * np.pi * t / 12)
def mu_profile(t):
    return 1 - gompertz_growth_rate(t, 0.01, 1, 1, 4)
rec_profile = random_profile(t0=0, tmax=24, nt=100, fmax=0.05)
rec.add_operator(Receiver(inducer=ahl, output=sfp1, a=0, b=100, K=1, n=2, profile=rec_profile))

Now we have constructed the circuit we need to run an assay containing some samples. The sample is driven by a metabolism which defines the dynamics of growth and gene expression profiles:

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

Next we create a set of samples associated to Flapjack media and strain, and containing our AHL at different concentrations

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

# Create list of samples    
samples = []
for conc in np.logspace(-6, 6, 12):
    sample = Sample(circuit=rec, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

Now we can create and run the assay:

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica receiver',
              description='Simulated receiver generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run(nsr=1e-3)

Plot the results:

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].groupby('Sample').plot(x='Time', y='Measurement', style='.', ax=ax)
plt.yscale('log')
len(m)

Upload the simulated data to flapjack

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])

In [ ]:
#vector = fj.get('vector', name='pREC')
#media = fj.get('media', name='Simulated media')
#strain = fj.get('strain', name='Simulated strain')
#signal = fj.get('signal', name='CFP')
#biomass_signal = fj.get('signal', name='OD')
#print(biomass_signal)

signal = fj.get('signal', name='CFP')
vector = fj.get('vector', name='Rec_1e-3_nsr')
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
cfp = fj.get('signal', name='CFP')
biomass_signal = fj.get('signal', name='OD')
analyte = fj.get('chemical', name='AHL')

char_receiver = Receiver(None, None, 0,0,0,0)
char_receiver.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=signal.id,
    biomass_signal=biomass_signal.id,
    n_gaussians=20,
    epsilon=0
)

In [ ]:
t = np.linspace(0, 24, 100)
print(f'a = {char_receiver.a}')
print('b = ', char_receiver.b)
print('K = ', char_receiver.K)
print('n = ', char_receiver.n)

plt.plot(t, char_receiver.profile(t))
plt.plot(t, rec_profile(t))
plt.legend(['Characterization', 'True'])

### Inverter

In [ ]:
dna = fj.get('dna', name='Inv_1e-3_nsr')
if len(dna)==0:
    dna = fj.create('dna', name='Inv_1e-3_nsr')
vector = fj.get('vector', name='Inv_1e-3_nsr')    
if len(vector)==0:
    vector = fj.create('vector', name='Inv_1e-3_nsr', dnas=dna.id)
    
inv = GeneticNetwork(vector=vector.id[0])

In [ ]:
def mu_profile(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 1)
inv_profile = random_profile(t0=0, tmax=24, nt=100, fmax=0.05)

In [ ]:
ahl = Supplement(name='AHL')
laci = Regulator('LacI', degradation_rate=2)
rec = Receiver(inducer=ahl, output=laci, a=0, b=100, K=1, n=2, profile=rec_profile)
inv.add_operator(rec)
inv.add_regulator(laci)

In [ ]:
cfp = fj.get('signal', name='CFP')
cfp = Reporter('CFP', signal_id=cfp.id[0])
inv.add_reporter(cfp)
inv.add_operator(Not(input=laci, output=cfp, a=100, b=0, K=1, n=2, profile=inv_profile))

In [ ]:
# Create list of samples    
samples = []
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
for conc in np.logspace(-6, 6, 12):
    sample = Sample(circuit=inv, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica inverter',
              description='Simulated inverter generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])

In [ ]:
signal = fj.get('signal', name='CFP')
receiver = fj.get('vector', name='Rec_1e-3_nsr')
inverter = fj.get('vector', name='Inv_1e-3_nsr')
media = fj.get('media', name='Loica')
strain = fj.get('strain', name='Loica strain')
biomass_signal = fj.get('signal', name='OD')
analyte = fj.get('chemical', name='AHL')

char_not = Not(input=None, output=None, a=100, b=0, K=1, n=2)
char_not.characterize(
    fj,
    receiver = receiver.id,
    inverter=inverter.id,
    media=media.id,
    strain=strain.id,
    signal=signal.id,
    biomass_signal=biomass_signal.id,
    n_gaussians=20,
    epsilon=0,
    gamma=2
)

In [ ]:
t = np.linspace(0, 24, 100)
plt.plot(t, char_not.profile(t), 'r--')
plt.plot(t, inv_profile(t), 'r')
plt.plot(t, rec_profile(t), 'g')
plt.plot(t, char_not.profile_A(t), 'g--')

print(char_not.a_A, char_not.b_A, char_not.K_A, char_not.n_A)
print(char_not.a, char_not.b, char_not.K, char_not.n) #, char_not.gamma)

In [ ]:
J = char_not.res.jac
H = J.T.dot(J)
C = np.linalg.inv(H) * np.sum(char_not.res.fun*char_not.res.fun) / 100
covar = np.diag(np.abs(C))
std = np.sqrt(covar)
print(std[:3])
print(char_not.n, char_not.K, char_not.b)
plt.imshow(C[:,:]); plt.colorbar()

In [ ]:
means = np.linspace(t.min(), t.max(), 20)
vars = [t.max()/20]*20 #res.x[:n_blobs]

profiles = []
for i in range(1000):
    rheights = np.random.multivariate_normal(char_not.res.x[3:], C[3:,3:])
    profile = np.zeros_like(t)
    for mean,var,height in zip(means, vars, rheights):
        gaussian = height * np.exp(-(t-mean)*(t-mean) / var / 2) / np.sqrt(2 * np.pi * var)
        profile += gaussian
    profiles.append(np.array(profile))
profiles = np.array(profiles)
mean = np.mean(profiles, axis=0)
std = np.std(profiles, axis=0)
plt.plot(t, mean, 'k')
plt.fill_between(t, mean-std, mean+std, color='k', alpha=0.2)

### Repressilator

In [ ]:
dna = fj.get('dna', name='Rep')
if len(dna)==0:
    dna = fj.create('dna', name='Rep')
vector = fj.get('vector', name='Rep')    
if len(vector)==0:
    vector = fj.create('vector', name='Rep', dnas=dna.id)
    
rep = GeneticNetwork(vector=vector.id[0])

In [ ]:
laci = Regulator(name='LacI', degradation_rate=1, init_concentration=5)
tetr = Regulator(name='TetR', degradation_rate=1)
ci = Regulator(name='cI', degradation_rate=1)
rep.add_regulator(laci)
rep.add_regulator(tetr)
rep.add_regulator(ci)

cfp = fj.get('signal', name='CFP')
yfp = fj.get('signal', name='YFP')
rfp = fj.get('signal', name='RFP')

sfp1 = Reporter(name='CFP', degradation_rate=1, signal_id=cfp.id[0])
rep.add_reporter(sfp1)
sfp2 = Reporter(name='YFP', degradation_rate=1, signal_id=yfp.id[0])
rep.add_reporter(sfp2)
sfp3 = Reporter(name='RFP', degradation_rate=1, signal_id=rfp.id[0])
rep.add_reporter(sfp3)

rep.add_operator(Not(input=ci, output=laci, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=laci, output=tetr, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=tetr, output=ci, a=100, b=0, K=1, n=2))

rep.add_operator(Not(input=ci, output=sfp1, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=laci, output=sfp2, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=tetr, output=sfp3, a=100, b=0, K=1, n=2))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

biomass_signal = fj.get('signal', name='OD')

sample = Sample(circuit=rep, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0]
               )
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica repressilator',
              description='Simulated repressilator generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
assay.measurements

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='YFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='RFP'].plot(x='Time', y='Measurement', ax=ax)

Upload simulated data to flapjack

In [ ]:
assay.upload(fj, study.id[0])

### Toggle switch

In [ ]:
dna = fj.create('dna', name='Toggle')
vector = fj.create('vector', name='Toggle', dnas=dna.id)
tog = GeneticNetwork(vector=vector.id[0])

laci = Regulator(name='LacI', degradation_rate=1, init_concentration=0.1)
ci = Regulator(name='cI', degradation_rate=1)
tog.add_regulator(laci)
tog.add_regulator(ci)

cfp = fj.get('signal', name='CFP')
yfp = fj.get('signal', name='YFP')
sfp1 = Reporter(name='CFP', degradation_rate=1, signal_id=cfp.id[0])
tog.add_reporter(sfp1)
sfp2 = Reporter(name='YFP', degradation_rate=1, signal_id=yfp.id[0])
tog.add_reporter(sfp2)

tog.add_operator(Not(input=laci, output=ci, a=10, b=0, K=1, n=2))
tog.add_operator(Not(input=ci, output=laci, a=10, b=0, K=1, n=2))

tog.add_operator(Not(input=ci, output=sfp1, a=10, b=0, K=1, n=2))
tog.add_operator(Not(input=laci, output=sfp2, a=10, b=0, K=1, n=2))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

biomass_signal = fj.get('signal', name='OD')
sample = Sample(circuit=tog, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0]
               )
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica toggle',
              description='Simulated toggle switch generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='YFP'].plot(x='Time', y='Measurement', ax=ax)

Upload simulated data to flapjack

In [ ]:
assay.upload(fj, study.id[0])

### Nor gate

In [ ]:
dna = fj.get('dna', name='Nor')
if len(dna)==0:
    dna = fj.create('dna', name='Nor')
vector = fj.get('vector', name='Nor')    
if len(vector)==0:
    vector = fj.create('vector', name='Nor', dnas=dna.id)
    
nor = GeneticNetwork(vector=vector.id[0])

Create a reporter and associate it with a Flapjack signal so we can record the behaviour of the circuit:

In [ ]:
cfp = fj.get('signal', name='CFP')
sfp1 = Reporter(name='CFP', degradation_rate=0, signal_id=cfp.id[0])

nor.add_reporter(sfp1)

Create and add a receiver operator to the circuit, linking it to an AHL supplement and the receptor we just created:

In [ ]:
ahl1 = Supplement(name='AHL1')
ahl2 = Supplement(name='AHL2')
nor.add_operator(Nor(input=[ahl1, ahl2], output=sfp1, alpha=[0.0001,1,1,1], a=[100,100], b=[1,1], K=[1,1], n=[2,2]))

Now we have constructed the circuit we need to run an assay containing some samples. The sample is driven by a metabolism which defines the dynamics of growth and gene expression profiles:

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)
    
metab = SimulatedMetabolism(biomass, growth_rate)

Next we create a set of samples associated to Flapjack media and strain, and containing our AHL at different concentrations

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

# Create list of samples    
samples = []
for conc1 in np.logspace(-3, 3, 6):
    for conc2 in np.logspace(-3,3,6):
        sample = Sample(circuit=nor, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
        # Add AHL to samples at given concentration
        sample.add_supplement(ahl1, conc1)
        sample.add_supplement(ahl2, conc2)
        samples.append(sample)

Now we can create and run the assay:

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.25,
              name='Loica nor',
              description='Simulated nor generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

Plot the results:

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

Upload the simulated data to flapjack

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])